<a href="https://colab.research.google.com/github/KendallScott/QTW/blob/main/Case%20Study%203/Case_Study_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import quopri
import codecs
import email
from gensim.parsing.preprocessing import STOPWORDS
from nltk import word_tokenize
import chardet
import os
import pandas as pd 
import numpy as np

In [34]:
file_list = []
for root, dirs, files in os.walk("/content/drive/MyDrive/SpamAssassinMessages/", topdown=False):
    for name in files:
        tmp = os.path.join(root,name)
        file_list.append(tmp)
    for item in dirs:
        print(item)
i=0

file_list_full = []
while i < len(file_list):
    file_list_full.append(file_list[i])
    i+=1

spam_2
spam
easy_ham_2
easy_ham
hard_ham


In [46]:
len(file_list_full)

9354

Option one-- sets the decoder dynamically, but still needs more work for pulling in information on other email types


In [53]:
i=0

contents = []
types = []
labels = []


for root, dirs, files in os.walk("/content/drive/MyDrive/SpamAssassinMessages/"):
    for name in files:
      
        #this may need to be adjusted-- 

        blob = open(file_list_full[i], 'rb').read()

        #determining the encoding of the email, so the open(os.path) portion doesn't hit an error downloading an email

        result = chardet.detect(blob)
        encoding_dynamic = result['encoding']

        #if 'unknown' in encoding_dynamic:
          #this needs to be setup to pull in the correct encoding when magic fails to identify the right encoding
         # encoding_dynamic='utf-8'

        with open(os.path.join(root,name), encoding=encoding_dynamic, errors='replace') as f:
            x = email.message_from_file(f)
            print('normal', file_list_full[i])
            i+=1
            
            if "multipart" in x.get_content_type():
                              # Clean up multipart email to raw string
                              content_parts = [part.get_payload().replace("\n", " ")
                                              for part in x.walk() if part.get_content_type() == "text/plain"]
                              content = "\n".join(content_parts)
                              contents.append(content)

                              types.append(x.get_content_type())

            elif "text/plain" in x.get_content_type():  #boy I hope all text messages are the same!
                # clean up multipart email to raw string
                contents.append(x.get_payload().replace("\n"," "))
                types.append(x.get_content_type())

            elif "html" in x.get_content_type():
                # clean up html email to raw string
                contents.append(x)
                types.append(x.get_content_type())

            if "ham" in root:
                labels.append(0)
            elif "spam" in root:
                labels.append(1)
            else:
                print("YOU HAVE A BIG PROBLEM--LABEL NOT FOUND")
                
        

Streaming output truncated to the last 5000 lines.
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0772.f6e3d3c433121335121efc42f2d81aa7
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0708.fe13d44640fb0f6e7d9d6a50c813a01f
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0706.ef90d85cf26d95101f9f5071ac457a81
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0753.45965735008c580a773f99eada080a27
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0736.0c8647e849c1d900b6af3a6c7024752d
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0774.cdd7992b853cadd1098b09224e74c728
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0843.6e5323ee268f72a690f5e5af60643e73
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0836.249730de164ba4999f1ab6ec2d3cb2d1
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0877.62f5636ba5885d1b92423169c83a35b9
normal /content/drive/MyDrive/SpamAssassinMessages/easy_ham/0863.3d

In [54]:
print('contents length: ', len(contents))
print('types length: ', len(types))
print('labels length: ', len(labels))
print('file list length: ', len(file_list_full))

contents length:  9354
types length:  9354
labels length:  9354
file list length:  9354


In [55]:
data = pd.DataFrame({"Text":contents,"Label":labels, "Type":types, "Filename": file_list_full})
data

,Text,Label,Type,Filename
0,SUBJECT: Urgent Concern Pl...,1,text/plain,/content/drive/MyDrive/SpamAssassinMessages/sp...
1,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
2,"As seen on NBC, CBS, CNN, and even Oprah! The ...",1,text/plain,/content/drive/MyDrive/SpamAssassinMessages/sp...
3,,1,multipart/mixed,/content/drive/MyDrive/SpamAssassinMessages/sp...
4,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
...,...,...,...,...
9349,"Hi XFS developers, When studying xfs code(2.4....",0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...
9350,My gut reaction would be to tell them to go an...,0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...
9351,* WORD A DAY * ---------...,0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...
9352,==============================================...,0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...


In [59]:
data[data['Type'].str.contains("html")]

,Text,Label,Type,Filename
1,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
4,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
6,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
7,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
8,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
...,...,...,...,...
9332,"[Return-Path, Received, Received, Received, Me...",0,text/html,/content/drive/MyDrive/SpamAssassinMessages/ha...
9334,"[Return-Path, Delivery-Date, Return-Path, Rece...",0,text/html,/content/drive/MyDrive/SpamAssassinMessages/ha...
9338,"[Return-Path, Delivered-To, Received, Received...",0,text/html,/content/drive/MyDrive/SpamAssassinMessages/ha...
9343,"[Return-Path, Delivered-To, Received, Received...",0,text/html,/content/drive/MyDrive/SpamAssassinMessages/ha...


In [60]:
data

,Text,Label,Type,Filename
0,SUBJECT: Urgent Concern Pl...,1,text/plain,/content/drive/MyDrive/SpamAssassinMessages/sp...
1,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
2,"As seen on NBC, CBS, CNN, and even Oprah! The ...",1,text/plain,/content/drive/MyDrive/SpamAssassinMessages/sp...
3,,1,multipart/mixed,/content/drive/MyDrive/SpamAssassinMessages/sp...
4,"[Return-Path, Delivery-Date, Received, Receive...",1,text/html,/content/drive/MyDrive/SpamAssassinMessages/sp...
...,...,...,...,...
9349,"Hi XFS developers, When studying xfs code(2.4....",0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...
9350,My gut reaction would be to tell them to go an...,0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...
9351,* WORD A DAY * ---------...,0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...
9352,==============================================...,0,text/plain,/content/drive/MyDrive/SpamAssassinMessages/ha...


In [61]:
from sklearn.feature_extraction.text import CountVectorizer

In [62]:
vectorizer = CountVectorizer()

In [63]:
out = vectorizer.fit_transform(data['Text'])

AttributeError: ignored

In [64]:
vectorizer.vocabulary_

AttributeError: ignored

In [ ]:
data.iloc[0,0]

In [ ]:
<unk>
            

In [ ]:
reverse={}
for word,value in vectorizer.vocabulary_.items():
    reverse[value]=word

In [ ]:
reverse[64792]

In [ ]:
for i in range(out[0].shape[1]):
    if out[0,i]>0:
        print(reverse[i], out[0,i])

In [ ]:
out[0,0]

In [ ]:
vectorizer.vocabulary_['unk']

In [ ]:
<unk> = 102
<start> = 101
<end>= 3
<pad> = 100
<cls>

In [ ]:
"<pad> <pad> <start> The <unk> car drove away <end>"

In [ ]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
word_counts = CountVectorizer()
v_data = word_counts.fit_transform(data['data'])
v_test = word_counts.transform(test['data'])

In [ ]:
v_data

In [ ]:
from sklearn.naive_bayes import ComplementNB, BernoulliNB, GaussianNB, CategoricalNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
comp = ComplementNB()
bern = BernoulliNB()
gaus = GaussianNB()
cat = CategoricalNB()
multi = MultinomialNB()
methods = {"Complement":comp,"Bernoulli":bern,"Gausian":gaus,"Multi":multi,"Cat":cat}
for i in methods.keys():
    try:
        methods[i].fit(v_data,data['target'])
        p = methods[i].predict(v_test)
        print(accuracy_score(test['target'],p), f"method={i}")
        print(classification_report(test['target'],p))
    except:
        print(f"{i} failed")


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)
ConfusionMatrixDisplay.from_predictions(test['target'],p,cmap='Blues')

In [ ]:
reverse={}
for word,value in word_counts.vocabulary_.items():
    reverse[value]=word